In [1]:
from filmModels import *
from dataPreprocess import *
from filmPostProcess import *
import os

c:\users\mouju\appdata\local\programs\python\python38\lib\site-packages\scipy\__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
label = 'visible_ice'
tab_params = ['proj_x_norm', 'proj_y_norm', 'depth_norm', 'interval_length_norm', 'year_norm', 'month_cyclic_norm']
label_predict = label + '_code'

film=False
chip_size = 64
train_max_epoch = 40
max_iterations = 10
print_train_progress = True
print_test_model = True
hidden_width=32 
hidden_nblocks=4
L2_param = 1e-5
seed = 42
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# path of csv containing tabular data
bh_file_path = 'data_stephen_fix_header.csv'


chips_path = r'C:\Users\mouju\Desktop\film\raster_chips\concat_80bh_hdslanczos_34_128_128'

meta_params = {'chip_size': chip_size, 'train_max_epoch': train_max_epoch, 'max_iterations': max_iterations, \
               'print_train_progress': print_train_progress, 'print_test_model':print_test_model, \
              'device':device, 'hidden_width': hidden_width, 'hidden_nblocks':hidden_nblocks, 'L2_param':L2_param, 'film':film, 'seed':seed}

In [3]:
# list of columns in the dataframe to normalize
list_cols = ['latitude', 'longitude', 'proj_x', 'proj_y', 'depth', 'interval_length', 'time']

df = pd.read_csv(bh_file_path, header=[0])
df, n_classes = prepare_df(df, list_cols, label)
df = crop_hds_discard_chips(df)

Dataframe has length 2837
Number of unique boreholes is 566
Latitude ranges from 68.36933 to 69.44755
Longitude ranges from -133.82843 to -132.89346
List of columns normalized: ['latitude_norm', 'longitude_norm', 'proj_x_norm', 'proj_y_norm', 'depth_norm', 'interval_length_norm', 'year_norm', 'month_cyclic_norm']
visible_ice: 'None' entries have been replaced by 'No visible ice'
visible_ice column entries has been ordered:
[NaN, 'Pure ice', 'No visible ice', 'High', 'Medium to high', 'Low']
Categories (5, object): ['No visible ice' < 'Low' < 'Medium to high' < 'High' < 'Pure ice']
with corresponding codes in visible_ice_code column:
[-1  4  0  3  2  1]
visible_ice: binary column generated
materials: 'ICE' and 'ice' entries has been standardized into 'Ice'
'material_ice' column generated
materials has been categorized into codes in materials_code
Null entries of visible_ice dropped
Number of dataframe rows: 399


In [4]:
loaded_dataset = filmDataset(chips_path, df, tab_params, label_predict, chip_size)

Dataset contains 34 channels


In [5]:
meta_params['n_channels'] = loaded_dataset.n_channels
meta_params['n_classes'] = n_classes
if n_classes == 1:
    meta_params['loss_fn'] = torch.nn.BCELoss()
else:
    meta_params['loss_fn'] = torch.nn.CrossEntropyLoss()
meta_params['full_dataset'] = loaded_dataset

In [6]:
train_loader, test_loader = split_data(loaded_dataset)

Batch size 20
Data split into training size 319 and testing size 80


In [7]:
# working on: where does mlp store its models

In [ ]:
results = iterate(meta_params, train_loader, test_loader)

epoch  1: running loss: 1.53959, validation loss: 1.54787
epoch  2: running loss: 1.50025, validation loss: 1.49170
epoch  3: running loss: 1.43002, validation loss: 1.38097
epoch  4: running loss: 1.35335, validation loss: 1.34956
epoch  5: running loss: 1.33308, validation loss: 1.37352
epoch  6: running loss: 1.32376, validation loss: 1.35133
epoch  7: running loss: 1.31670, validation loss: 1.34342
epoch  8: running loss: 1.30910, validation loss: 1.33391
epoch  9: running loss: 1.30385, validation loss: 1.34292
epoch 10: running loss: 1.29380, validation loss: 1.31796
epoch 11: running loss: 1.28936, validation loss: 1.31776
epoch 12: running loss: 1.28095, validation loss: 1.29886
epoch 13: running loss: 1.26965, validation loss: 1.29657
epoch 14: running loss: 1.25599, validation loss: 1.26991
epoch 15: running loss: 1.25336, validation loss: 1.27980
epoch 16: running loss: 1.24633, validation loss: 1.24312
epoch 17: running loss: 1.23476, validation loss: 1.25206
epoch 18: runn

In [ ]:
display_results(meta_params, results)

In [ ]:
## draw heatmap

In [ ]:
# df2 = df.dropna(subset=['visible_ice'])